In [1]:
!nvidia-smi
!which python | grep DYY

Sun Sep 15 21:17:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   42C    P0              54W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Model

In [2]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class SinousEmbedding(nn.Module):
#     def __init__(self, dim) -> None:
#         super().__init__()
#         assert dim%2==0,NotImplementedError()
#         self.angles = (1000.**(-2/dim))**torch.arange(1,dim//2+1,1,dtype=torch.float).cuda()
#         self.angles.requires_grad_(False)
#     def forward(self,x):
#         angles = torch.einsum('m,i->im',self.angles,x.float())
#         return torch.cat((torch.sin(angles),torch.cos(angles)),dim=1)

# class DDPM(nn.Module):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__(*args, **kwargs)
#         self.in_size = 28 * 28
#         self.t_embedding_dim = 256
#         self.t_embedding = SinousEmbedding(dim=self.t_embedding_dim)
#         self.up = nn.ModuleList([
#             nn.Sequential(
#                 nn.Linear(784+self.t_embedding_dim,64),
#                 nn.ReLU(),
#             ),
#             nn.Sequential(
#                 nn.Linear(64,32),
#                 nn.ReLU(),
#             ),
#             # nn.Sequential(
#             #     nn.Linear(256,256),
#             #     # nn.LeakyReLU(0.1),
#             # ),
#         ])
#         self.middle = nn.ModuleList([
#             nn.Linear(32,32),
#             # nn.LeakyReLU(0.1),
#         ])
#         self.down= nn.ModuleList([
#             nn.Sequential(
#                 nn.Linear(32,32),
#                 nn.ReLU(),
#             ),
#             # nn.Sequential(
#             #     nn.Linear(256,256),
#             #     # nn.LeakyReLU(0.1),
#             # ),
#             nn.Sequential(
#                 nn.Linear(32,64),
#                 nn.ReLU(),
#             ),
#         ])
#         self.end_mlp = nn.Linear(64,784)
#         self.apply_init()

#     def apply_init(self):
#         for m in self.modules():
#             if isinstance(m, nn.Linear):
#                 nn.init.xavier_normal_(m.weight)
#                 nn.init.constant_(m.bias, 0)

#     def forward(self,x,t):
#         x = x.reshape(-1,784)
#         ttensor = self.t_embedding(t) # [batch, 256]
#         batch = x.shape[0]
#         xc = x.clone()
#         ups = []
#         x = torch.cat((x,ttensor),dim=-1)
#         for ly in self.up:
#             x = ly(x)
#             ups.append(x.clone())
#         for ly in self.middle:
#             x = ly(x)
#         for ly in self.down:
#             x = ly(x) + ups.pop()

#         x = self.end_mlp(x)
#         x = (x + xc)
#         return x

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SinousEmbedding(nn.Module):
    def __init__(self, dim) -> None:
        super().__init__()
        assert dim%2==0,NotImplementedError()
        self.angles = (500.**(-2/dim))**torch.arange(1,dim//2+1,1,dtype=torch.float).cuda()
        self.angles.requires_grad_(False)
    def forward(self,x):
        angles = torch.einsum('m,i->im',self.angles,x.float())
        return torch.cat((torch.sin(angles),torch.cos(angles)),dim=1)

class F_x_t(nn.Module):

    def __init__(self,in_channels,out_channels,out_size,kernel_size=3,t_shape=64) -> None:
        super().__init__()
        # self.t_channels = out_channels // 2
        # self.conv_channels = out_channels - self.t_channels
        self.t_channels = out_channels
        self.conv_channels = out_channels
        self.conv = nn.Conv2d(in_channels, self.conv_channels, kernel_size=kernel_size, padding=kernel_size//2)
        self.out_size = out_size
        self.fc = nn.Linear(t_shape, self.t_channels)
        # self.fc = nn.Embedding(t_shape, self.t_num)

    def forward(self, x, t):
        if self.t_channels == 0:
            return self.conv(x)
        # return torch.cat([self.conv(x),self.fc(t).unsqueeze(-1).unsqueeze(-1).expand(t.shape[0], self.t_channels, self.out_size, self.out_size)],dim=1).relu()
        return (self.conv(x) + self.fc(t).unsqueeze(-1).unsqueeze(-1).expand(t.shape[0], self.t_channels, self.out_size, self.out_size)).relu()

class DDPM(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.t_embedding_dim = 32
        self.t_embedding = SinousEmbedding(dim=self.t_embedding_dim)
        self.up= nn.ModuleList([
            F_x_t(in_channels=1,out_channels=32,out_size=32,kernel_size=3,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=32,out_channels=128,out_size=16,kernel_size=3,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=128,out_channels=128,out_size=8,kernel_size=3,t_shape=self.t_embedding_dim),
        ])
        self.middle = nn.ModuleList([
            nn.Identity()
        ])
        self.down= nn.ModuleList([
            F_x_t(in_channels=128,out_channels=128,out_size=4,kernel_size=3,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=128,out_channels=128,out_size=8,kernel_size=3,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=128,out_channels=32,out_size=16,kernel_size=3,t_shape=self.t_embedding_dim),
        ])
        self.end_mlp = nn.Conv2d(32,1,kernel_size=3,padding=1)
        self.apply_init()
    
    def apply_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self,x,t):
        x = x.reshape(-1,1,28,28)
        x = F.pad(x,(2,2,2,2),mode='constant',value=0)
        ttensor = self.t_embedding(t) # [batch, 256]
        batch = x.shape[0]
        # xc = x.clone()
        ups = []
        for ly in self.up:
            x = ly(x,ttensor)
            ups.append(x.clone()) # append: 28x28, 14x14
            x = nn.AvgPool2d(2)(x)
        for ly in self.middle:
            x = ly(x)
        for ly in self.down:
            x = ly(x,ttensor)
            x = nn.Upsample(scale_factor=2)(x) + ups.pop()
        x = self.end_mlp(x)
        x = x[:,:,2:30,2:30]
        return x.reshape(batch,28*28)

# Train

In [4]:
import sys
import os

parent_dir = os.path.abspath('/home/zhh24/DeepLearning')

sys.path.append(parent_dir)
print('appended',parent_dir)

import utils

from tqdm import tqdm
import torch
import torch.nn.functional as F
import torchvision.utils

device = 'cuda' if torch.cuda.is_available() else 'cpu'

mnist = utils.MNIST(batch_size=512)
train_loader = mnist.train_dataloader
valid_loader = mnist.valid_dataloader
T=200
beta1=1e-4 # variance of lowest temperature
betaT=2e-2 # variance of highest temperature
# step = torch.log(torch.tensor(betaT/beta1))/(T-1)
# betas = beta1 * torch.exp(step*torch.arange(T,dtype=torch.float).to(device))
step = (betaT-beta1)/(T-1)
betas = torch.arange(beta1,betaT+step,step).to(device)

alphas = 1-betas
alpha_bars = alphas.clone()
for i in range(1,T):
    alpha_bars[i] *= alpha_bars[i-1]
print(alpha_bars)
print(alphas)
sqrt = torch.sqrt
sigmas = sqrt(betas)


@torch.no_grad()
def sample(model:DDPM,save_dir):
    x = torch.randn([100,784]).to(device)
    for t in range(T-1,-1,-1):
        sigmaz = torch.randn_like(x)*sigmas[t]
        if t==0:
            sigmaz = 0
        x = (x-(1-alphas[t])/(sqrt(1-alpha_bars[t]))*model(x,t*torch.ones(x.shape[0],dtype=torch.long,device=device)))/(sqrt(alphas[t]))+sigmaz
        # x = torch.clamp(x,0,1)
    grid = torchvision.utils.make_grid(post_process(x).reshape(-1,1,28,28).cpu(), nrow=10)
    torchvision.utils.save_image(grid, save_dir)

@torch.no_grad()
def visualize(model,save_dir):
    x = torch.randn([10,784]).to(device)
    x_history = []
    for t in range(T-1,-1,-1):
        sigmaz = torch.randn_like(x)*((betas[t])**0.5).to(device)
        if t==0:
            sigmaz = 0
        x = (x-(1-alphas[t])/(sqrt(1-alpha_bars[t]))*model(x,t*torch.ones(x.shape[0],dtype=torch.long,device=device)))/(sqrt(alphas[t]))+sigmaz
        # x = torch.clamp(x,0,1)
        x_history.append(x)
    # print('cat.shape',torch.cat(x_history,dim=0).shape)
    grid = torchvision.utils.make_grid(post_process(torch.cat(x_history,dim=0)[3::4,...]).reshape(-1,1,28,28).cpu(), nrow=10)
    torchvision.utils.save_image(grid, save_dir)
    print('Saved visualize to',os.path.abspath(save_dir))

def pre_process(x):
    # do the logit transform
    # return (torch.log(x+1e-3)-torch.log(1-x+1e-3))
    return (x+1)/2

def post_process(x):
    # return torch.sigmoid(x)
    return x*2-1

def train(epochs,model:DDPM,optimizer,eval_interval=5):
    for epoch in range(epochs):
        model.train()
        with tqdm(train_loader) as bar:
            losses = []
            for x,_ in bar:
                x = pre_process(x.to(device))
                epss = torch.randn_like(x).reshape(-1,784).to(device)
                ts = torch.randint(0,T,(x.shape[0],),device=device,dtype=torch.long)
                alpha_tbars = alpha_bars[ts]
                value = (sqrt(alpha_tbars).reshape(-1,1,1,1)*x).reshape(-1,784)+sqrt(1-alpha_tbars).reshape(-1,1)*epss
                out = model(value,ts) # [batch,784]
                # loss = ((epss-out).pow(2).mean(dim=-1) * (betas[ts])/(2*alphas[ts]*(1-alpha_tbars))).sum(dim=0)
                loss = (epss-out).pow(2).mean(dim=-1).mean(dim=0)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())
                bar.set_description('epoch {}, loss {:.4f}'.format(epoch,sum(losses)/len(losses)))
        model.eval()
        with torch.no_grad():
            with tqdm(valid_loader) as bar:
                mses = []
                losses = []
                for x,_ in bar:
                    x = pre_process(x.to(device))
                    epss = torch.randn_like(x).reshape(-1,784).to(device)
                    ts = torch.randint(0,T,(x.shape[0],),device=device,dtype=torch.long)
                    alpha_tbars = alpha_bars[ts]
                    value = (sqrt(alpha_tbars).reshape(-1,1,1,1)*x).reshape(-1,784)+sqrt(1-alpha_tbars).reshape(-1,1)*epss
                    out = model(value,ts)
                    mse = F.mse_loss(epss,out)
                    mses.append(mse.item())
                    # loss = ((epss-out).pow(2).mean(dim=-1) * (betas[ts])/(2*alphas[ts]*(1-alpha_tbars))).sum(dim=0)
                    loss = (epss-out).pow(2).mean(dim=-1).mean(dim=0)
                    losses.append(loss.item())
                    bar.set_description('epoch {}, MSE {:.4f}, [Valid] {:.4f}'.format(epoch,sum(mses)/len(mses),sum(losses)/len(losses)))
                    
        if epoch % eval_interval == 0:
            visualize(model,save_dir=os.path.join('./samples',f'diffuse_epoch_{epoch}.png'))
            sample(model,save_dir=os.path.join('./samples',f'sample_epoch_{epoch}.png'))
            torch.save(model,os.path.join('./samples',f'epoch_{epoch}.pt'))

if __name__ == '__main__':
    model = DDPM().to(device)
    print('Number parameters of the model:', sum(p.numel() for p in model.parameters()))
    print('Model strcuture:',model)
    optimizer = torch.optim.Adam(model.parameters(),lr=5e-3)
    os.makedirs('./samples',exist_ok=True)
    sample(model,save_dir=os.path.join('./samples',f'init.png'))
    visualize(model,save_dir=os.path.join('./samples',f'init_visualize.png'))
    train(200,model,optimizer,eval_interval=5)

appended /home/zhh24/DeepLearning
tensor([0.9999, 0.9997, 0.9994, 0.9990, 0.9985, 0.9979, 0.9972, 0.9964, 0.9955,
        0.9945, 0.9934, 0.9922, 0.9909, 0.9895, 0.9881, 0.9865, 0.9848, 0.9830,
        0.9812, 0.9792, 0.9771, 0.9750, 0.9728, 0.9704, 0.9680, 0.9655, 0.9629,
        0.9602, 0.9574, 0.9545, 0.9516, 0.9485, 0.9454, 0.9422, 0.9389, 0.9355,
        0.9320, 0.9285, 0.9249, 0.9212, 0.9174, 0.9135, 0.9096, 0.9056, 0.9015,
        0.8974, 0.8932, 0.8889, 0.8845, 0.8801, 0.8756, 0.8711, 0.8664, 0.8618,
        0.8570, 0.8522, 0.8474, 0.8425, 0.8375, 0.8325, 0.8274, 0.8223, 0.8171,
        0.8118, 0.8066, 0.8012, 0.7959, 0.7905, 0.7850, 0.7795, 0.7740, 0.7684,
        0.7628, 0.7572, 0.7515, 0.7458, 0.7400, 0.7342, 0.7284, 0.7226, 0.7168,
        0.7109, 0.7050, 0.6991, 0.6931, 0.6872, 0.6812, 0.6752, 0.6692, 0.6632,
        0.6571, 0.6511, 0.6450, 0.6390, 0.6329, 0.6268, 0.6207, 0.6147, 0.6086,
        0.6025, 0.5964, 0.5903, 0.5842, 0.5782, 0.5721, 0.5660, 0.5600, 0.5539,
      

epoch 0, loss 3.4624:   1%|█                                                                                                      | 1/94 [00:00<00:17,  5.20it/s]

Saved visualize to /home/zhh24/samples/init_visualize.png


epoch 0, MSE 1.0364, [Valid] 1.0364: 100%|███████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00,  9.49it/s]


Saved visualize to /home/zhh24/samples/diffuse_epoch_0.png


...

Saved visualize to /home/zhh24/samples/diffuse_epoch_30.png


epoch 35, MSE 0.1318, [Valid] 0.1318: 100%|██████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00,  9.42it/s]


Saved visualize to /home/zhh24/samples/diffuse_epoch_35.png


epoch 36, loss 0.1302:  26%|█████████████████████████▊                                                                           | 24/94 [00:03<00:09,  7.43it/s]

: 